# Forklift Detection - Webcam

In [1]:
# Import necessary packages
import cv2
import numpy as np
from ultralytics import YOLO


In [2]:
# Load the YOLO model
model = YOLO("yolomodels/best_forklift.pt")


In [3]:

# Function to calculate the Euclidean distance between two points
def euclidean_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

# Function to determine if a person is very close to a forklift
def is_close(person_box, forklift_box, threshold=350):
    person_center = ((person_box[0] + person_box[2]) / 2, (person_box[1] + person_box[3]) / 2)
    forklift_center = ((forklift_box[0] + forklift_box[2]) / 2, (forklift_box[1] + forklift_box[3]) / 2)
    distance = euclidean_distance(person_center, forklift_center)
    return distance < threshold


In [ ]:
# Open the webcam
cap = cv2.VideoCapture(0)

In [ ]:
import cv2
import numpy as np

# Set frame skip value
frame_skip = 1 # Process every 5th frame

# Initialize frame counter
frame_count = 0

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Skip frames
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
    
    # Run inference on the frame
    results = model(frame)
    
    # Extract the bounding boxes for detected objects
    persons = []
    forklifts = []
    drivers = []
    alert_triggered = False
    
    for result in results:
        for obj in result.boxes.data:
            x1, y1, x2, y2, conf, cls = obj[:6]
            class_name = result.names[int(cls)]
            if class_name == 'person':
                persons.append((x1, y1, x2, y2))
            elif class_name == 'forklift':
                forklifts.append((x1, y1, x2, y2))
            elif class_name == 'driver':
                drivers.append((x1, y1, x2, y2))
            
            # Draw bounding box
            if class_name == 'person':
                color = (0, 255, 0)  # Green for person
            elif class_name == 'forklift':
                color = (255, 0, 0)  # Blue for forklift
            elif class_name == 'driver':
                color = (0, 255, 255)  # Yellow for driver
                
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Put label with confidence
            label = f"{class_name} {conf:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Check for close proximity between persons and forklifts if a driver is detected
    #if drivers:  # Only proceed if at least one driver is detected
    for person in persons:
        for forklift in forklifts:
            if is_close(person, forklift):
                alert_triggered = True
                print("ALERT: Person is very close to a forklift while the driver is present!")
                cv2.putText(frame, "ALERT: Person very close to a forklift!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                break  # No need to check further once an alert is triggered
    
    # Display the frame
    cv2.imshow('Video', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [6]:
# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
